# 출력 파서
- 개념: 언어 모델(LLM)의 출력을 더 *유용하고 구조화된 형태로 변환하는* 컴포넌트
- 출력파서의 역할:
    - LLM의 출력을 받아 더 적합한 형식으로 변환
    - 구조화된 데이터 생성에 매우 유용
    - LangChain 프레임워크에서 다양한 종류의 출력 데이터를 파싱하고 처리
- 주요 특징:
    - 다양성: LangChain은 많은 종류의 출력 파서를 제공
    - 스트리밍 지원: 많은 출력 파서들이 스트리밍을 지원
    - 확장성: 최소한의 모듈부터 복잡한 모듈까지 확장 가능한 인터페이스를 제공
- 출력 파서의 이점:
    - 구조화: LLM의 자유 형식 텍스트 출력을 구조화된 데이터로 변환
    - 일관성: 출력 형식을 일관되게 유지하여 후속 처리를 용이하게 함
    - 유연성: 다양한 출력 형식 *(JSON, 리스트, 딕셔너리 등)*으로 변환 가능

# 3-1. Pydantic 출력 파서(PydanticOutputParser)*

- `PydanticOutputParser`: 언어 모델의 출력을 **더 구조화된 정보로 변환** 하는 데 도움.

    PydanticOutputParser (이는 대부분의 OutputParser에 해당 되기 함) 에는 주로 두 가지 핵심 메서드가 구현 되어야 함.

- `get_format_instructions()`: 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침(instruction) 을 제공. 이 지침에 따라 언어 모델은 출력을 구조화하고, 이를 특정 데이터 모델에 맞게 변환할 수 있음.

- `parse()`: 언어 모델의 출력(문자열로 가정)을 받아들여 이를 특정 구조로 분석하고 변환. Pydantic와 같은 도구를 사용하여, 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환.

In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
# LangSmith 추적을 설정
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [1]:
# 실시간 출력을 위한 import
from langchain_teddynote.messages import stream_response

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

llm = ChatOpenAI(temperature = 0, model_name = "gpt-4o")

In [9]:
# 이메일 본문 예시
email_conversation = """From: 김도현 (dohyun.kim@hybeent.com)
To: 박서윤 (seoyoon.park@jypent.com)
Subject: 신규 아티스트 협업 및 미팅 일정 제안

안녕하세요, 박서윤 매니저님,

저는 하이브 엔터테인먼트의 김도현 이사입니다. 최근 귀사의 신규 아티스트 라인업 및 글로벌 진출 전략에 대한 소식을 접하고 큰 관심을 갖게 되어 연락드립니다. 하이브 엔터테인먼트는 음악 및 아티스트 매니지먼트 분야에서 풍부한 경험을 보유하고 있으며, 글로벌 시장에서의 다양한 협업 기회를 모색하고 있습니다.

이에 따라, 귀사 신규 아티스트 관련 브로슈어 및 주요 활동 계획에 대한 자료를 요청드립니다. 특히 음악 스타일, 콘셉트, 해외 진출 계획에 대한 세부 정보를 확인할 수 있다면, 보다 구체적인 협력 방안을 제안드릴 수 있을 것 같습니다.

또한, 협업 가능성을 보다 심도 있게 논의하기 위해 다음 주 목요일(3월 7일) 오후 2시에 미팅을 제안드립니다. 귀사 사무실에서 직접 논의할 수 있을까요?

검토 후 긍정적인 회신 부탁드립니다. 감사합니다.

김도현
이사
하이브 엔터테인먼트
"""

### 출력 파서를 사용하지 않은 경우

In [10]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(temperature = 0, model_name="gpt-4o")
chain = prompt | llm
answer = chain.stream({"email_conversation": email_conversation})
output = stream_response(answer, return_output = True)

이메일의 중요한 내용은 다음과 같습니다:

1. 김도현 이사(하이브 엔터테인먼트)가 박서윤 매니저(JYP 엔터테인먼트)에게 신규 아티스트 협업을 제안.
2. JYP의 신규 아티스트 라인업 및 글로벌 진출 전략에 관심을 표명.
3. JYP의 신규 아티스트 관련 브로슈어 및 주요 활동 계획 자료 요청.
4. 협업 가능성을 논의하기 위해 3월 7일 목요일 오후 2시에 미팅 제안.
5. 미팅 장소는 JYP 사무실로 제안.

In [11]:
print(output)

이메일의 중요한 내용은 다음과 같습니다:

1. 김도현 이사(하이브 엔터테인먼트)가 박서윤 매니저(JYP 엔터테인먼트)에게 신규 아티스트 협업을 제안.
2. JYP의 신규 아티스트 라인업 및 글로벌 진출 전략에 관심을 표명.
3. JYP의 신규 아티스트 관련 브로슈어 및 주요 활동 계획 자료 요청.
4. 협업 가능성을 논의하기 위해 3월 7일 목요일 오후 2시에 미팅 제안.
5. 미팅 장소는 JYP 사무실로 제안.


### 출력 파서를 사용하는 경우

In [12]:
class EmailSummary(BaseModel):
    person: str = Field(description = "메일을 보낸 사람")
    email: str = Field(description = "메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description = "메일 제목")
    summary: str = Field(description = "메일 본문을 요약한 텍스트")
    date: str = Field(description = "메일 본문에 언급된 미팅 날짜와 시간")
    
# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object = EmailSummary)

In [13]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [16]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())


In [17]:
chain = prompt | llm

In [18]:
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됩니다.
output = stream_response(response, return_output=True)

```json
{
    "person": "김도현",
    "email": "dohyun.kim@hybeent.com",
    "subject": "신규 아티스트 협업 및 미팅 일정 제안",
    "summary": "김도현 이사는 JYP 엔터테인먼트의 신규 아티스트 라인업과 글로벌 진출 전략에 관심을 가지고 협업을 제안하며, 관련 자료를 요청하고 3월 7일 오후 2시에 미팅을 제안합니다.",
    "date": "3월 7일 오후 2시"
}
```

In [19]:
# 결과 파싱
structured_output = parser.parse(output)
print(structured_output)

person='김도현' email='dohyun.kim@hybeent.com' subject='신규 아티스트 협업 및 미팅 일정 제안' summary='김도현 이사는 JYP 엔터테인먼트의 신규 아티스트 라인업과 글로벌 진출 전략에 관심을 가지고 협업을 제안하며, 관련 자료를 요청하고 3월 7일 오후 2시에 미팅을 제안합니다.' date='3월 7일 오후 2시'


### parser가 추가된 체인 생성

In [21]:
chain = prompt | llm | parser

In [22]:
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요."
    }
)

response

EmailSummary(person='김도현', email='dohyun.kim@hybeent.com', subject='신규 아티스트 협업 및 미팅 일정 제안', summary='김도현 이사는 JYP 엔터테인먼트의 신규 아티스트 라인업 및 글로벌 진출 전략에 관심을 가지고 협업을 제안하며, 관련 자료 요청과 함께 3월 7일 오후 2시에 미팅을 제안합니다.', date='3월 7일 오후 2시')

### with_structured_output()

In [23]:
llm_with_structered = ChatOpenAI(
    temperature=0, model_name="gpt-4o"
).with_structured_output(EmailSummary)

In [24]:
# invoke() 함수를 호출하여 결과를 출력합니다.
answer = llm_with_structered.invoke(email_conversation)
answer

EmailSummary(person='김도현', email='dohyun.kim@hybeent.com', subject='신규 아티스트 협업 및 미팅 일정 제안', summary='하이브 엔터테인먼트의 김도현 이사가 JYP 엔터테인먼트의 박서윤 매니저에게 신규 아티스트 협업을 제안하며, 관련 자료 요청 및 협업 논의를 위한 미팅을 제안하는 내용입니다.', date='3월 7일 오후 2시')

# 3-2. 콤마 구분자 출력 파서*

In [1]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()
# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    # 주제에 대한 다섯 가지를 나열하라는 템플릿
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],  # 입력 변수로 'subject' 사용
    # 부분 변수로 형식 지침 사용
    partial_variables={"format_instructions": format_instructions},
)

# ChatOpenAI 모델 초기화
model = ChatOpenAI(temperature=0)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser

In [3]:
# 체인 호출 실행
chain.invoke({"subject": "[세계 여행 명소"})

['에펠탑', '마추픽추', '피렌체 대성당', '타지 마할', '그랜드 캐년']

# 3-3. 구조화된 출력 파서
- LLM에 대한 답변을 dict 형식으로 구성하고 key/value 쌍으로 갖는 여러 필드를 반환하고자 할 때 사용
- GPT, Claude 모델보다 인텔리전스가 낮은(parameter 수가 낮은) 모델) 에 유용

- `ResponseSchema` 클래스를 사용하여 사용자의 질문에 대한 답변과 사용된 소스(웹사이트)에 대한 설명을 포함하는 응답 스키마를 정의
    - cf) 스키마: 데이터 구조를 정의하는 설계도 - 어떤 데이터가 어떤 형식으로 저장되고, 어떤 필드를 가져야 하는지 미리 정해놓는 것
- `StructuredOutputParser`를 response_schemas를 사용하여 초기화하여, 정의된 응답 스키마에 따라 출력을 구조화

In [4]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

- `ResponseSchema` 클래스를 사용하여 사용자의 질문에 대한 답변과 사용된 소스(웹사이트)에 대한 설명을 포함하는 응답 스키마를 정의합니다.
- `StructuredOutputParser`를 `response_schemas`를 사용하여 초기화하여, 정의된 응답 스키마에 따라 출력을 구조화합니다.


In [5]:
# 사용자의 질문에 대한 답변
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
]
# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

이제 응답이 어떻게 포맷되어야 하는지에 대한 지시사항이 포함된 문자열을 받게 되며(schemas), 정의된 스키마를 프롬프트에 삽입합니다.

In [6]:
# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)

In [7]:
model = ChatOpenAI(temperature=0)  # ChatOpenAI 모델 초기화
chain = prompt | model | output_parser  # 프롬프트, 모델, 출력 파서를 연결

In [10]:
# 질문*
chain.invoke({"question": "대한민국 광복은 언제인가요?"})

{'answer': '대한민국 광복은 1945년 8월 15일에 이루어졌습니다.',
 'source': 'https://ko.wikipedia.org/wiki/%EA%B4%91%EB%B3%B5%EC%9D%80_%EC%96%B4%EC%A0%9C%EC%9D%B8%EA%B0%80'}

`chain.stream` 메소드*

In [11]:
for s in chain.stream({"question": "이순신의 업적은 무엇인가요?"}):
    # 스트리밍 출력
    print(s)

{'answer': '이순신은 조선시대의 무신 이순신 장군으로서, 임진왜란에서 일본의 침략을 막고 전략적으로 승리를 거둬 업적을 남겼습니다.', 'source': 'https://ko.wikipedia.org/wiki/%EC%9D%B4%EC%88%9C%EC%8B%A0'}


# 3-4. JSON 출력 파서

**[참고]**

`JSON (JavaScript Object Notation)` 은 데이터를 저장하고 구조적으로 전달하기 위해 사용되는 경량의 데이터 교환 포맷입니다. 웹 개발에서 매우 중요한 역할을 하며, 서버와 클라이언트 간의 통신을 위해 널리 사용됩니다. JSON은 읽기 쉽고, 기계가 파싱하고 생성하기 쉬운 텍스트를 기반으로 합니다.

JSON의 기본 구조
JSON 데이터는 이름(키)과 값의 쌍으로 이루어져 있습니다. 여기서 "이름"은 문자열이고, "값"은 다양한 데이터 유형일 수 있습니다. JSON은 두 가지 기본 구조를 가집니다:

- `객체`: 중괄호 {}로 둘러싸인 키-값 쌍의 집합입니다. 각 키는 콜론 :을 사용하여 해당하는 값과 연결되며, 여러 키-값 쌍은 쉼표 ,로 구분됩니다.
- `배열`: 대괄호 []로 둘러싸인 값의 순서 있는 목록입니다. 배열 내의 값은 쉼표 ,로 구분됩니다.

```json
{
  "name": "John Doe",
  "age": 30,
  "is_student": false,
  "skills": ["Java", "Python", "JavaScript"],
  "address": {
    "street": "123 Main St",
    "city": "Anytown"
  }
}
```

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [13]:
# OpenAI 객체를 생성합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

원하는 출력 구조를 정의합니다.


In [14]:
# 원하는 데이터 구조를 정의합니다.
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

`JsonOutputParser`를 사용하여 파서를 설정하고, 프롬프트 템플릿에 지시사항을 주입합니다.

In [15]:
# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "\uc8fc\uc81c\uc5d0 \ub300\ud55c \uac04\uacb0\ud55c \uc124\uba85", "title": "Description", "type": "string"}, "hashtags": {"description": "\ud574\uc2dc\ud0dc\uadf8 \ud615\uc2dd\uc758 \ud0a4\uc6cc\ub4dc(2\uac1c \uc774\uc0c1)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [16]:
# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 체인을 구성합니다.
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
answer = chain.invoke({"question": question})

In [17]:
# 타입을 확인합니다.
type(answer)

dict

In [18]:
# answer 객체를 출력합니다.
answer

{'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 주로 인간 활동에 의해 발생하는 온실가스 배출이 주요 원인입니다. 이는 극지방의 빙하가 녹고 해수면이 상승하며, 기후 패턴이 변화하여 극단적인 기상 현상이 증가하는 등 다양한 환경적 영향을 초래합니다.',
 'hashtags': '#지구온난화 #기후변화'}

**Pydantic 을 사용하지 않고 `JsonOutputParser` 를 사용**

Pydantic 없이도 이 기능을 사용할 수 있습니다. 이 경우 JSON을 반환하도록 요청하지만, 스키마가 어떻게 되어야 하는지에 대한 구체적인 정보는 제공하지 않습니다.

- 간단한 JSON 변환 → JsonOutputParser 사용
- 스키마 기반 데이터 검증 & 구조화 → PydanticOutputParser 사용

In [19]:
# 질의 작성
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
response = chain.invoke({"question": question})

# 출력을 확인합니다.
print(response)

{'description': '지구 온난화는 대기 중 온실가스 농도의 증가로 인해 지구의 평균 기온이 상승하는 현상입니다. 이는 주로 화석 연료의 연소, 산림 파괴, 산업 활동 등 인간의 활동에 의해 촉발됩니다. 지구 온난화는 기후 변화, 해수면 상승, 극단적인 기상 현상 증가 등 다양한 환경적 문제를 야기합니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#환경문제', '#해수면상승']}


# 3-5. PandasDataFrameOutputParser

**Pandas DataFrame**은 Python 프로그래밍 언어에서 널리 사용되는 데이터 구조로, 데이터 조작 및 분석을 위한 강력한 도구입니다. DataFrame은 구조화된 데이터를 효과적으로 다루기 위한 포괄적인 도구 세트를 제공하며, 이를 통해 데이터 정제, 변환 및 분석과 같은 다양한 작업을 수행할 수 있습니다.

이 **출력 파서**는 사용자가 임의의 Pandas DataFrame을 지정하여 해당 DataFrame에서 데이터를 추출하고, 이를 형식화된 사전(dictionary) 형태로 조회할 수 있게 해주는 LLM(Large Language Model) 기반 도구입니다.


In [20]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [29]:
# ChatOpenAI 모델 초기화 (gpt-3.5-turbo 모델 사용을 권장합니다)
model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

`format_parser_output` 함수는 파서 출력을 사전 형식으로 변환하고 출력 형식을 지정하는 데 사용됩니다. 

In [30]:
# 출력 목적으로만 사용됩니다.
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    # 파서 출력의 키들을 순회합니다.
    for key in parser_output.keys():
        # 각 키의 값을 딕셔너리로 변환합니다.
        parser_output[key] = parser_output[key].to_dict()
    # 예쁘게 출력합니다.
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

- `titanic.csv` 데이터를 읽어온 뒤 DataFrame 을 로드하여 `df` 변수에 할당합니다.
- PandasDataFrameOutputParser를 사용하여 DataFrame을 파싱합니다.


In [31]:
# 원하는 Pandas DataFrame을 정의합니다.
df = pd.read_csv("./data/titanic.csv")
df.head()

,Passengerid,Age,Fare,Sex,sibsp,zero,zero.1,zero.2,zero.3,zero.4,...,zero.12,zero.13,zero.14,Pclass,zero.15,zero.16,Embarked,zero.17,zero.18,2urvived
0,1,22.0,7.2500,0,1,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
1,2,38.0,71.2833,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0.0,0,0,1
2,3,26.0,7.9250,1,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,1
3,4,35.0,53.1000,1,1,0,0,0,0,0,...,0,0,0,1,0,0,2.0,0,0,1
4,5,35.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0


In [32]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = PandasDataFrameOutputParser(dataframe=df)

# 파서의 지시사항을 출력합니다.
print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [33]:
print(f"'{df.columns}'")  # 첫 번째 컬럼명 확인

'Index(['Passengerid', 'Age', 'Fare', 'Sex', 'sibsp', 'zero', 'zero.1',
       'zero.2', 'zero.3', 'zero.4', 'zero.5', 'zero.6', 'Parch', 'zero.7',
       'zero.8', 'zero.9', 'zero.10', 'zero.11', 'zero.12', 'zero.13',
       'zero.14', 'Pclass', 'zero.15', 'zero.16', 'Embarked', 'zero.17',
       'zero.18', '2urvived'],
      dtype='object')'


컬럼에 대한 값을 조회하는 예제입니다.

In [34]:
# 열 작업 예시입니다.
df_query = "Age column 을 조회해 주세요."


# 프롬프트 템플릿을 설정합니다.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{question}\n",
    input_variables=["question"],  # 입력 변수 설정
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },  # 부분 변수 설정
)

# 체인 생성
chain = prompt | model | parser

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 출력
format_parser_output(parser_output)

{'Age': {0: 22.0,
         1: 38.0,
         2: 26.0,
         3: 35.0,
         4: 35.0,
         5: 28.0,
         6: 54.0,
         7: 2.0,
         8: 27.0,
         9: 14.0,
         10: 4.0,
         11: 58.0,
         12: 20.0,
         13: 39.0,
         14: 14.0,
         15: 55.0,
         16: 2.0,
         17: 28.0,
         18: 31.0,
         19: 28.0,
         20: 35.0,
         21: 34.0,
         22: 15.0,
         23: 28.0,
         24: 8.0,
         25: 38.0,
         26: 28.0,
         27: 19.0,
         28: 28.0,
         29: 28.0,
         30: 40.0,
         31: 28.0,
         32: 28.0,
         33: 66.0,
         34: 28.0,
         35: 42.0,
         36: 28.0,
         37: 21.0,
         38: 18.0,
         39: 14.0,
         40: 40.0,
         41: 27.0,
         42: 28.0,
         43: 3.0,
         44: 19.0,
         45: 28.0,
         46: 28.0,
         47: 28.0,
         48: 28.0,
         49: 18.0,
         50: 7.0,
         51: 21.0,
         52: 49.0,
         

첫 번째 행을 검색하는 예시입니다.

In [35]:
# 행 조회 예시입니다.
df_query = "Retrieve the first row."

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 결과 출력
format_parser_output(parser_output)

{'0': {'2urvived': 0.0,
       'Age': 22.0,
       'Embarked': 2.0,
       'Fare': 7.25,
       'Parch': 0.0,
       'Passengerid': 1.0,
       'Pclass': 3.0,
       'Sex': 0.0,
       'sibsp': 1.0,
       'zero': 0.0,
       'zero.1': 0.0,
       'zero.10': 0.0,
       'zero.11': 0.0,
       'zero.12': 0.0,
       'zero.13': 0.0,
       'zero.14': 0.0,
       'zero.15': 0.0,
       'zero.16': 0.0,
       'zero.17': 0.0,
       'zero.18': 0.0,
       'zero.2': 0.0,
       'zero.3': 0.0,
       'zero.4': 0.0,
       'zero.5': 0.0,
       'zero.6': 0.0,
       'zero.7': 0.0,
       'zero.8': 0.0,
       'zero.9': 0.0}}


특정 열에서 일부 행의 평균을 검색하는 작업 예제입니다.

In [36]:
# row 0 ~ 4의 평균 나이를 구합니다.
df["Age"].head().mean()

31.2

In [37]:
# 임의의 Pandas DataFrame 작업 예시, 행의 수를 제한합니다.
df_query = "Retrieve the average of the Ages from row 0 to 4."

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 결과 출력
print(parser_output)

{'mean': 31.2}


다음은 요금(Fare) 에 대한 평균 가격을 산정하는 예시입니다.

In [38]:
df.head()

,Passengerid,Age,Fare,Sex,sibsp,zero,zero.1,zero.2,zero.3,zero.4,...,zero.12,zero.13,zero.14,Pclass,zero.15,zero.16,Embarked,zero.17,zero.18,2urvived
0,1,22.0,7.2500,0,1,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
1,2,38.0,71.2833,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0.0,0,0,1
2,3,26.0,7.9250,1,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,1
3,4,35.0,53.1000,1,1,0,0,0,0,0,...,0,0,0,1,0,0,2.0,0,0,1
4,5,35.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0


In [41]:
# 잘못 형식화된 쿼리의 예시입니다.
df_query = "Calculate average `Fare` rate." ## 따옴표 때문인가? *

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# 결과 출력
print(parser_output)

{'mean': 33.28108563789152}


In [42]:
# 결과 검증
df["Fare"].mean()

33.28108563789152

# 3-6. 날짜 형식 출력 파서

**참고**

| 형식 코드 | 설명                | 예시          |
|------------|---------------------|---------------|
| %Y         | 4자리 연도          | 2024          |
| %y         | 2자리 연도          | 24            |
| %m         | 2자리 월            | 07            |
| %d         | 2자리 일            | 04            |
| %H         | 24시간제 시간       | 14            |
| %I         | 12시간제 시간       | 02            |
| %p         | AM 또는 PM          | PM            |
| %M         | 2자리 분            | 45            |
| %S         | 2자리 초            | 08            |
| %f         | 마이크로초 (6자리)  | 000123        |
| %z         | UTC 오프셋          | +0900         |
| %Z         | 시간대 이름         | KST           |
| %a         | 요일 약어           | Thu           |
| %A         | 요일 전체           | Thursday      |
| %b         | 월 약어             | Jul           |
| %B         | 월 전체             | July          |
| %c         | 전체 날짜와 시간     | Thu Jul  4 14:45:08 2024 |
| %x         | 전체 날짜           | 07/04/24      |
| %X         | 전체 시간           | 14:45:08      |

In [43]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 날짜 및 시간 출력 파서
output_parser = DatetimeOutputParser()
output_parser.format = "%Y-%m-%d"

# 사용자 질문에 대한 답변 템플릿
template = """Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    },  # 지침을 템플릿에 적용
)

# 프롬프트 내용을 출력
prompt

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%d'.\n\nExamples: 1721-11-08, 1514-06-27, 1692-04-25\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:')

체인을 생성합니다.


In [45]:
# Chain 을 생성합니다.
chain = prompt | ChatOpenAI() | output_parser

# 체인을 호출하여 질문에 대한 답변을 받습니다.
output = chain.invoke({"question": "NAVER가 창업한 연도"})

결과를 출력합니다.


In [ ]:
# 결과를 문자열로 변환
output.strftime("%Y-%m-%d") #틀렸음 (1999-06-02)

'1999-06-18'

# 3-7. 열거형 출력 파서

LangChain의 EnumOutputParser는 언어 모델의 출력을 미리 정의된 열거형(Enum) 값 중 하나로 파싱하는 도구입니다. 이 파서의 주요 특징과 사용법은 다음과 같습니다.

## 주요 특징

- **열거형 파싱**: 문자열 출력을 미리 정의된 Enum 값으로 변환합니다.
- **타입 안전성**: 파싱된 결과가 반드시 정의된 Enum 값 중 하나임을 보장합니다.
- **유연성**: 공백이나 줄바꿈 문자를 자동으로 처리합니다.

## 사용 방법

EnumOutputParser는 언어 모델의 출력에서 유효한 Enum 값을 추출하는 데 유용합니다. 이를 통해 출력 데이터의 일관성을 유지하고 예측 가능성을 높일 수 있습니다. 파서를 사용하려면, 미리 정의된 Enum 값을 설정하고 해당 값을 기준으로 문자열 출력을 파싱합니다.

In [47]:
from langchain.output_parsers.enum import EnumOutputParser

- `enum` 모듈을 사용하여 `Colors` 클래스를 정의합니다.
- `Colors` 클래스는 `Enum`을 상속받으며, `RED`, `GREEN`, `BLUE` 세 가지 색상 값을 가집니다.


In [49]:
from enum import Enum


class Colors(Enum):
    pink = "분홍색"
    yellow = "노랑색"
    black = "검정색"

`EnumOutputParser` 를 생성합니다.

In [50]:
# EnumOutputParser 인스턴스 생성
parser = EnumOutputParser(enum=Colors)

In [51]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template(
    """다음의 물체는 어떤 색깔인가요?

Object: {object}

Instructions: {instructions}"""
    # 파서에서 지시사항 형식을 가져와 부분적으로 적용합니다.
).partial(instructions=parser.get_format_instructions())

# 체인을 생성합니다.
chain = prompt | ChatOpenAI() | parser

- `chain.invoke` 함수를 사용하여 input에 대한 정보를 요청합니다.


In [ ]:
# 체인 실행*
response = chain.invoke({"object": "브로콜리"})
print(response)

OutputParserException: Response '초록색' is not one of the expected values: ['분홍색', '노랑색', '검정색']

<에러 설명>
- EnumOutputParser가 특정한 Enum 값만 허용하도록 설정되어 있는데, LLM이 초록색을 반환함
- 하지만 Colors Enum에는 초록색이 존재하지 않음
- LLM이 "초록색"을 출력했지만, 기대하는 값은 [분홍색, 노랑색, 검정색]이기 때문에 ValueError 발생

In [ ]:
# 체인을se = chain.invoke({"object": "바나나"})
print(response)

Colors.yellow


결과를 확인합니다.


In [54]:
# 출력의 타입을 확인합니다.
type(response)

<enum 'Colors'>

In [55]:
# 출력의 값을 확인합니다.
response.value

'노랑색'

# 3-8. 출력 수정 파서

`OutputFixingParser`는 출력 파싱 과정에서 발생할 수 있는 오류를 자동으로 수정하는 기능을 제공합니다. 이 파서는 기본적으로 다른 파서, 예를 들어 `PydanticOutputParser` 를 래핑하고, 이 파서가 처리할 수 없는 형식의 출력이나 오류를 반환할 경우, 추가적인 LLM 호출을 통해 오류를 수정하도록 설계되었습니다.

이러한 접근 방식의 핵심은, 첫 번째 시도에서 스키마를 준수하지 않는 결과가 나올 경우, `OutputFixingParser` 가 자동으로 형식이 잘못된 출력을 인식하고, 이를 수정하기 위한 새로운 명령어와 함께 모델에 다시 제출한다는 것입니다. 이 과정에서, 수정을 위한 명령어는 오류를 정확히 지적하고, 올바른 형식으로 데이터를 재구성할 수 있도록 구체적인 지시를 포함해야 합니다.

예를 들어, `PydanticOutputParser` 를 사용하여 특정 데이터 스키마를 준수하는 출력을 생성하려고 했지만, 일부 필드가 누락되었거나 데이터 유형이 잘못된 경우가 발생할 수 있습니다. 이때 `OutputFixingParser` 는 다음 단계로, 해당 오류를 수정하는 지시를 포함한 새로운 요청을 LLM에 제출합니다. LLM은 이 지시를 바탕으로 오류를 수정한 새로운 출력을 생성하게 됩니다.


In [58]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List


class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

(참고) 다음의 Cell 은 오류가 발생해야 **정상** 입니다.

In [59]:
# 잘못된 형식을 일부러 입력
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

# 잘못된 형식으로 입력된 데이터를 파싱하려고 시도
parser.parse(misformatted)

# 오류 출력

OutputParserException: Invalid json output: {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}

<에러 설명>
- 싱글쿼트(') 사용: JSON에서는 모든 키와 문자열 값이 더블쿼트(")로 감싸져야 함

`OutputFixingParser` 를 사용하여 잘못된 형식을 바로 잡도록 하겠습니다.


In [60]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [61]:
# 잘못된 형식의 출력
misformatted

"{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

In [62]:
# OutputFixingParser 를 사용하여 잘못된 형식의 출력을 파싱
actor = new_parser.parse(misformatted)

In [63]:
# 파싱된 결과
actor

Actor(name='Tom Hanks', film_names=['Forrest Gump'])